In [124]:
import pandas as pd
import numpy as np

In [24]:
#llama a los datasets
df15=pd.read_csv('data/players_15.csv')
df16=pd.read_csv('data/players_16.csv')
df17=pd.read_csv('data/players_17.csv')
df18=pd.read_csv('data/players_18.csv')
df19=pd.read_csv('data/players_19.csv')
df20=pd.read_csv('data/players_20.csv')
df21=pd.read_csv('data/players_21.csv')
df22=pd.read_csv('data/players_22.csv')

C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (104) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (81,82,83,84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
#bucle para añadir columna version en todos los datasets
for i,df in zip(range(2015,2023),(df15,df16,df17,df18,df19,df20,df21,df22)):
    df['año_version']=i

In [26]:
#fusiono todos los datasets en uno
frames = [df15,df16,df17,df18,df19,df20,df21,df22]
df = pd.concat(frames).reset_index()

In [27]:
#eliminamos columnas de url que no aportan información
df = df[df.columns.drop(list(df.filter(regex='url')))]

## Agrupacion de posiciones

In [28]:
#funcion transformación en lista
def lista_posicion(x):
    return (x.replace(' ','').split(','))

In [29]:
#transforma valores de columna player_positions en una lista
df['player_positions']=df['player_positions'].apply(lambda x: lista_posicion(x))

In [30]:
#defino listas para agrupar posiciones por 4 categorias
pos_defensas=['LCB','RCB','CB','RB','LB','RWB','LWB']
pos_medios=['LW','LCM','RCM','CAM','RW','LDM','CDM','CM','RDM','RM','RAM','LAM','LM']
pos_delanteros=['CF','ST','RS','LS','RF']

In [31]:
#agrupo posiciones en posiciones genéricas    
for i in df.index:
            if len(df.at[i,'player_positions'])==1:
                if df.at[i,'player_positions'][0]=='GK':
                    df.at[i,'player_positions']='portero'
                if df.at[i,'player_positions'][0] in pos_defensas:
                    df.at[i,'player_positions']='defensa'
                elif df.at[i,'player_positions'][0] in pos_medios:
                    df.at[i,'player_positions']='centrocampista'
                elif df.at[i,'player_positions'][0] in pos_delanteros:
                    df.at[i,'player_positions']='delantero'
            else:
                if df.at[i,'player_positions'][0] in pos_defensas:
                    df.at[i,'player_positions']='defensa'
                elif df.at[i,'player_positions'][0] in pos_medios:
                    df.at[i,'player_positions']='centrocampista'
                elif df.at[i,'player_positions'][0] in pos_delanteros:
                    df.at[i,'player_positions']='delantero'
                elif df.at[i,'player_positions'][0]=='GK':
                    df.at[i,'player_positions']='portero'

In [33]:
#comprobación de dataset
df.groupby('player_positions')['sofifa_id'].count()

player_positions
centrocampista    58700
defensa           46218
delantero         21369
portero           15792
Name: sofifa_id, dtype: int64

## definir variable objetivo

In [45]:
def progresion_anual(row):
    #Calcula primero la primera y ultima version en que apareció el jugador en el juego. 
    #Despúes calcula la progresión porcentual que tendrá el overall en la versión siguiente.
    #El dateframe debe llamarse 'df'
    #Ojo! calculando por el dataframe segmento de la primera división tarda en ejecucarse unos 28.1s. El df total son 13 min
    #Sample Input: df.iloc[8].squeeze()
    #Sample Output: -0.022727272727272728
    
    
    firstfifa=df[df['sofifa_id']==row['sofifa_id']].nsmallest(1, 'año_version')['año_version'].to_list()[0]
    lastfifa=df[df['sofifa_id']==row['sofifa_id']].nlargest(1, 'año_version')['año_version'].to_list()[0]
    try:
        if row['año_version'] in range(firstfifa,lastfifa):
            value_fut=df[(df['sofifa_id']==row['sofifa_id'])&(df['año_version']==(row['año_version']+1))]['overall'].to_list()[0]
            value_act=df[(df['sofifa_id']==row['sofifa_id'])&(df['año_version']==row['año_version'])]['overall'].to_list()[0]
            return (value_fut-value_act)/value_act
        else:
            return None
    except:
        pass

In [46]:
%%time
df['progresion_anual']=df.apply(progresion_anual, axis=1)

Wall time: 11min 43s


In [48]:
#quito valores nulos de progresion anual
df=df[df['progresion_anual'].notnull()]

In [50]:
df=df.reset_index()

## Transformación columnas overall por posiciones

In [52]:
#df solo con las columnas a transformar
posiciones=df[["ls","st","rs","lw","lf","cf","rf","rw", 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm',
     'lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb',"gk"]]

In [79]:
#funcion para limpiar valores
def limp_posiciones(x):
    x.replace("+"," ")
    return x.split(" ")[0]


In [83]:
#limpiar en df
for i in posiciones.columns:
    df[i]=df[i].astype(str)
    df[i]=df[i].apply(limp_posiciones)

ValueError: invalid literal for int() with base 10: '8+'

## Añadir columnas nuevas

In [88]:
#Agrupa por Equipo y año versión(temporada) y añade el total en cada jugador
df['overall_team/year'] = df.groupby(['club_team_id', 'año_version'])['overall'].transform('sum')

In [89]:
#Crea columna nº de equipos por liga/año
df['teams_in_league/year'] = df.groupby(['league_name', 'año_version'])['club_team_id'].transform('nunique')

In [90]:
#Crea columna media liga/año
df['mean_league/year'] = df.groupby(['league_name', 'año_version'])['overall'].transform('mean')

In [98]:
#Crea columna media equipo/año
df['mean_team/year'] = df.groupby(['club_team_id', 'año_version'])['overall'].transform('mean')

## Columnas ranking equipos

In [92]:
ranking=pd.read_csv('data/ranking_equipos2.csv')

In [94]:
ranking.drop('Unnamed: 0',axis=1,inplace=True)

In [101]:
df=df.merge(ranking,how="left",on="club_name")

## Gestión de missing values

In [108]:
df.isna().sum().sort_values(ascending=False).head(32)

nation_team_id               81070
nation_position              81070
goalkeeping_speed            78114
player_traits                45640
mentality_composure          22682
passing                       9633
dribbling                     9633
shooting                      9633
pace                          9633
defending                     9633
physic                        9633
club_joined                   6510
league_level                  1142
value_eur                      946
club_contract_valid_until      840
club_jersey_number             840
club_position                  840
league_name                    840
club_name                      840
club_team_id                   840
overall_team/year              840
teams_in_league/year           840
mean_league/year               840
mean_team/year                 840
wage_eur                       834
link                           188
ranking                        133
puntos                         133
lf                  

In [119]:
#eliminamos columnas no relevantes
df.drop(["club_loaned_from","nation_jersey_number",'player_tags','release_clause_eur'],axis=1,inplace=True)

KeyError: "['club_loaned_from' 'nation_jersey_number' 'player_tags'\n 'release_clause_eur'] not found in axis"

In [ ]:
df.drop(['level_0','index'],axis=1,inplace=True)

In [141]:
#reemplazo valor nan = 0
df[['goalkeeping_speed','player_traits','club_team_id','club_jersey_number',
   'club_contract_valid_until', 'value_eur','wage_eur','pace','shooting',
   'passing','dribbling','defending','club_joined','physic','shooting',
  'league_level','mentality_composure','club_joined','mean_team/year','mean_league/year','teams_in_league/year','overall_team/year','puntos','ranking']] = df[['goalkeeping_speed','player_traits','club_team_id','club_jersey_number',
   'club_contract_valid_until', 'value_eur','wage_eur','pace','shooting',
   'passing','dribbling','defending','club_joined','physic','shooting',
  'league_level','mentality_composure','club_joined','mean_team/year','mean_league/year','teams_in_league/year','overall_team/year','puntos','ranking']].replace(np.nan, 0)

In [129]:
#Replace NaNs por Jugador libre
df[['club_name','league_name']] = df[['club_name','league_name']].replace(np.nan, "jugador_libre")

In [130]:
#Transformar por Nation_Team_Id por 0 y 1:
df['nation_team_id'] = df['nation_team_id'].replace(np.nan, 0)
df['nation_team_id'] = df['nation_team_id'].apply(lambda x: 0 if x==0 else 1)

In [131]:
#Transformar por Club_Position por 0 y 1:
df['club_position'] = df['club_position'].replace(np.nan, 0)
def titular(x):
    if x=='SUB':
        return (0)
    elif x=="RES":
        return (0)
    else:
        return (1)
df['club_position'] = df['club_position'].apply(titular)

In [132]:
#Transformar por nation_position por 0 y 1:
df['nation_position'] = df['nation_position'].replace(np.nan, 0)
df['nation_position'] = df['nation_position'].apply(lambda x: 0 if x==0 else 1)

In [143]:
#drop la columna link porque no aportar información relevante
df.drop("link",axis=1,inplace=True)

In [151]:
df[['puntos','ranking']]=df[['puntos','ranking']].replace('No disponible',0)

In [152]:
#clubes sin liga asignada
df[df['puntos']==0]['club_name'].unique()

array(['Olympiacos CFP', 'Levante Unión Deportiva',
       'Montpellier Hérault SC', 'Demir Grup Sivasspor',
       'Sport-Club Freiburg', 'Club de Foot Montréal',
       'Évian Thonon Gaillard FC', 'En Avant de Guingamp',
       'Unión Deportiva Almería', 'Club Atlético Tigre', 'CD Nacional',
       'Unión Deportiva Las Palmas', 'İstanbul Başakşehir FK',
       'Stade Malherbe Caen', 'Kardemir Karabükspor', 'Sporting Lokeren',
       'FC Sochaux-Montbéliard', 'İttifak Holding Konyaspor',
       'Derby County', "CD O'Higgins", 'Associação Académica de Coimbra',
       'RCD Mallorca', 'Clube Sport Marítimo',
       'Club Atlético Independiente', 'Belenenses SAD', 'Ipswich Town',
       'Club Deportivo Guadalajara', 'Stade Lavallois Mayenne FC',
       'Charlton Athletic', 'Inverness Caledonian Thistle',
       'Gençlerbirliği SK', 'Trapani', 'SD Ponferradina',
       'AS Nancy Lorraine', 'RC Recreativo de Huelva', 'CD Numancia',
       'Modena', 'Club de Deportes Cobresal', 'Club Deport

## exportación fichero

In [154]:
df.to_csv('data\df_cleaned.csv',index=False)